In [1]:
using LinearAlgebra
using Optim
using HDF5

using QuantumOptics
basis = NLevelBasis(2)

include("../LiPoSID.jl")

using Statistics

using Dates, Random

┌ Warning: Package LinearSolve does not have KrylovKit in its dependencies:
│ - If you have LinearSolve checked out for development and have
│   added KrylovKit as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with LinearSolve
└ Loading KrylovKit into LinearSolve from project dependency, future warnings for LinearSolve are suppressed.


In [2]:
Threads.nthreads()

70

Fidelity for qubit can be written:

$F(\rho, \sigma) = \operatorname{tr}(\rho \sigma) + 2\sqrt{\det(\rho) \det(\sigma)}$

In [3]:
function infidelity_norm(ρ, σ)
    abs(1 - real(tr(ρ * σ)) - 2*sqrt(abs(det(ρ)*det(σ))))
end

infidelity_norm (generic function with 1 method)

In [4]:
function read_timeevolution(file_name, state, γ)
    h5open(file_name, "r") do file
        ρᵧ = read(file[state][string(γ)])
        t = ρᵧ["t"]
        ρ₀₀ = ρᵧ["p0"]; Re_ρ₀₁ = ρᵧ["s_re"];  Im_ρ₀₁ = ρᵧ["s_im"]
        ρ_series = []
        t_series = []

        for i in 1:length(t)
            ρᵢ= [ ρ₀₀[i]                      Re_ρ₀₁[i] + im * Im_ρ₀₁[i]
                  Re_ρ₀₁[i] - im * Im_ρ₀₁[i]  1 - ρ₀₀[i]                 ]
            push!(ρ_series, convert(Matrix{ComplexF64}, ρᵢ))
            push!(t_series, convert(Float64, t[i]))
        end
        return(t_series, ρ_series)
    end
end

# Define the Pauli matrices
σˣ = [0 1; 1 0]
σʸ = [0 im; -im 0]
σᶻ = [1 0; 0 -1]

# Define the basis elements
fᴷ₁ = σˣ / 2
fᴷ₂ = σʸ / 2
fᴷ₃ = σᶻ / 2

# Check orthogonality and normalization
@assert tr(fᴷ₁ * fᴷ₂) ≈ 0
@assert tr(fᴷ₁ * fᴷ₃) ≈ 0
@assert tr(fᴷ₂ * fᴷ₃) ≈ 0
@assert tr(fᴷ₁ * fᴷ₁) ≈ 1 / 2
@assert tr(fᴷ₂ * fᴷ₂) ≈ 1 / 2
@assert tr(fᴷ₃ * fᴷ₃) ≈ 1 / 2

fᴼᴺᴮ = [fᴷ₁, fᴷ₂, fᴷ₃]

# Function to calculate Dc
function Dc(ρ, t, H, C)
    U = (H * ρ - ρ * H) / im
    D = sum(C .* [2 * fᵢ * ρ * fⱼ' - ρ * fⱼ' * fᵢ - fⱼ' * fᵢ * ρ for fᵢ in fᴼᴺᴮ, fⱼ in fᴼᴺᴮ]) / 2
    return U + D
end

function construct_H(params)
    ϵ, h_Re, h_Im = params[1:3]
    return [
        ϵ               h_Re + im * h_Im
        h_Re - im * h_Im -ϵ
    ] / 2
end

function construct_C(params)
    m_Re = reshape(params[4:12], (3, 3))
    m_Im = reshape(params[13:21], (3, 3))
    M = m_Re + im * m_Im
    return M * M'
end


# Define the objective function for optimization
function kossak_obj(params, ρ, t)
    H = construct_H(params)
    C = construct_C(params)

    obj = 0.0
    for i in 3:length(ρ)
        ρ1 = ρ[i]
        ρ2 = ρ[i - 2] + (t[i] - t[i - 1]) * (Dc(ρ[i], t[i], H, C) + 4 * Dc(ρ[i - 1], t[i - 1], H, C) + Dc(ρ[i - 2], t[i - 2], H, C)) / 3
        obj += infidelity_norm(ρ1,ρ2)
    end
    return obj
end

kossak_obj (generic function with 1 method)

In [5]:
# Define initial parameters
initial_params = [0.0, 0.0, 0.0,  # H parameters: ϵ, h_Re, h_Im
                  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,  # m_Re parameters
                  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]; # m_Im parameters

initial_params .+= 1;

In [6]:
# Define the objective function wrapper
function objective(params)

    objGEXY = kossak_obj(params, ρᵍ, tᵍ) + kossak_obj(params, ρᵉ, tᵉ) + kossak_obj(params, ρˣ, tˣ) + kossak_obj(params, ρʸ, tʸ)

    return objGEXY
end

objective (generic function with 1 method)

In [7]:
file_name = "../DATA/ALL_GAMMAS_B4_D10.h5"
γ = ["0.079477", "0.25133", "0.79477", "2.5133", "7.9477", "25.133", "79.477", "251.33"]
date_and_time_string = string(Dates.format(now(), "yyyy-u-dd_at_HH-MM"))
tests_data_file_name = "KOSSAK_UNCONSTR_BFGS_MULT_100_trn4_tst20_" * date_and_time_string * ".h5"
tests_dir = ""

ρᵍ₀ = [1 0; 0 0] # state to measure initial distance from
dodeca_10_states = ["D" * string(n) for n in 1:10]
basis_states = ["B" * string(n) for n in 1:4]
train_states = basis_states
test_states = dodeca_10_states

# Pre-read the data for all γ values
data = Dict()
for γᵢ in γ
    data[γᵢ] = (
        read_timeevolution(file_name, "B1", γᵢ),
        read_timeevolution(file_name, "B2", γᵢ),
        read_timeevolution(file_name, "B3", γᵢ),
        read_timeevolution(file_name, "B4", γᵢ)
    )
end

function perform_optimization(γᵢ, lock)
    #println("γ =  " * γᵢ)
    γᶠ = parse(ComplexF64, γᵢ)

    (tᵍ, ρᵍ), (tᵉ, ρᵉ), (tˣ, ρˣ), (tʸ, ρʸ) = data[γᵢ]

    function objective(params)
        objGEXY = kossak_obj(params, ρᵍ, tᵍ) + kossak_obj(params, ρᵉ, tᵉ) + kossak_obj(params, ρˣ, tˣ) + kossak_obj(params, ρʸ, tʸ)
        return objGEXY
    end

    obj = Inf
    best_params = []

    Threads.@threads for run in 1:100
        print("|γ =  " * γᵢ*":", run)
        initial_params = rand(21) .* (260 * 2) .- 260
        result = optimize(objective, initial_params, method = BFGS())#, g_tol = 1e-10)
        optimized_params = result.minimizer
        if objective(optimized_params) < obj
            lock() do
                if objective(optimized_params) < obj
                    obj = objective(optimized_params)
                    best_params = optimized_params
                end
            end
        end
    end

    optimized_params = best_params
    optimized_H = construct_H(optimized_params)
    optimized_C = construct_C(optimized_params)

    Hˢⁱᵈ = convert.(ComplexF64, optimized_H)
    Cˢⁱᵈ = convert.(ComplexF64, optimized_C)
    effective_Lindblad = LiPoSID.get_lindblad_operators(Cˢⁱᵈ, fᴼᴺᴮ)

    h5open(tests_dir * tests_data_file_name, "cw") do fid
        γ_group = create_group(fid, "gamma_" * γᵢ) # create coupling group
        γ_group["H"] = convert.(ComplexF64, Hˢⁱᵈ)
        γ_group["C"] = convert.(ComplexF64, Cˢⁱᵈ)
    end

    FminStates = []
    FmedianStates = []
    FmeanStates = []

    Threads.@threads for state in test_states
        print(state * " ")

        tₛ, ρₛ = read_timeevolution(file_name, state, γᵢ)
        ρₛ = convert(Vector{Matrix{ComplexF64}}, ρₛ)
        ρᵗˢᵗ = [DenseOperator(basis, Hermitian(ρₜ)) for ρₜ in ρₛ]
        tᵗˢᵗ = convert.(Float64, tₛ)
        ρₒ = DenseOperator(basis, ρₛ[1])
        dt = tᵗˢᵗ[2] - tᵗˢᵗ[1]
        tᵉⁿᵈ = tᵗˢᵗ[end]
        effective_Lindblad_ops = [DenseOperator(basis, j) for j in effective_Lindblad]
        tout, ρ_t_kossak = timeevolution.master(tᵗˢᵗ, ρₒ, DenseOperator(basis, Hˢⁱᵈ), effective_Lindblad_ops)
        ρˢⁱᵈ = [ρₜ.data for ρₜ in ρ_t_kossak]
        F = LiPoSID.fidelity_series(basis, ρₛ, ρˢⁱᵈ)

        h5open(tests_dir * tests_data_file_name, "cw") do fid
            γ_group = open_group(fid, "gamma_" * γᵢ) # open coupling group
            init_state_group = create_group(γ_group, state) # create initial state group
            init_state_group["Fidelity"] = convert.(Float64, F)
        end

        FminState = minimum(F)
        FmedianState = median(F)
        FmeanState = mean(F)

        lock() do
            push!(FminStates, FminState)
            push!(FmedianStates, FmedianState)
            push!(FmeanStates, FmeanState)
        end
    end

    F_median_value = median(FmedianStates)
    F_min_value = minimum(FminStates)

    println()
    println("Mimimal fidelity for " * γᵢ * ": ", F_min_value)
    println("Median fidelity for " * γᵢ * ": ", F_median_value)
end

lock = ReentrantLock()
Threads.@threads for γᵢ in γ
    perform_optimization(γᵢ, lock)
end

println(tests_data_file_name)


|γ =  251.33:1|γ =  0.79477:1|γ =  0.079477:1|γ =  7.9477:1|γ =  0.25133:1|γ =  2.5133:1|γ =  79.477:1|γ =  25.133:1

LoadError: TaskFailedException

[91m    nested task error: [39mMethodError: objects of type ReentrantLock are not callable
    Stacktrace:
     [1] [0m[1mmacro expansion[22m
    [90m   @ [39m[90m./[39m[90m[4mIn[7]:44[24m[39m[90m [inlined][39m
     [2] [0m[1m(::var"#88#threadsfor_fun#17"{String, ReentrantLock, var"#objective#16"{Vector{Any}, Vector{Any}, Vector{Any}, Vector{Any}, Vector{Any}, Vector{Any}, Vector{Any}, Vector{Any}}, UnitRange{Int64}})[22m[0m[1m([22m[90monethread[39m::[0mBool[0m[1m)[22m
    [90m   @ [39m[35mMain[39m [90m./[39m[90m[4mthreadingconstructs.jl:85[24m[39m
     [3] [0m[1m#invokelatest#2[22m
    [90m   @ [39m[90m./[39m[90m[4messentials.jl:716[24m[39m[90m [inlined][39m
     [4] [0m[1minvokelatest[22m
    [90m   @ [39m[90m./[39m[90m[4messentials.jl:714[24m[39m[90m [inlined][39m
     [5] [0m[1mmacro expansion[22m
    [90m   @ [39m[90m./[39m[90m[4mthreadingconstructs.jl:90[24m[39m[90m [inlined

D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 

Mimimal fidelity for 0.25133: 0.9995188642141652

Median fidelity for 0.25133: 0.9999049135214064